# Data preparation, processing and visualization

At this point, let's remind our core aim: to analyse some interesting relationships (esp. rent prices and distance from metro) based on data scraped from sReality.cz. 

Now, having the scraped massive dataset, our goal is to analyse and visualize underlaying relationships of chosen variables. We attempt to do so by (a) matplotlib and (b) geopandas tools. Nevertheless, before we can do so, the data need to be inspected and adjusted.

## I. Getting familiar with the data

First, we prepare the environment for analysis and get familiar with the structure and variables of data.

In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv("dataset.csv", sep = ';') #uploading our scraped data
data.head(4)

,Unnamed: 0,Adresa,Aktualizace:,Balkón:,Bankomat:,Bazén:,Bezbariérový:,Bus MHD:,Celková cena:,Cena:,...,Večerka:,Vlak:,Vlastnictví:,Voda:,Vybavení:,Výtah:,Výška stropu:,Zlevněno:,Škola:,Školka:
0,0,"Dobrovolného, Praha 9 - Černý Most Panorama",['Dnes'],NaN,"['Bankomat České spořitelny', ' (542 m)']",NaN,NaN,"['Generála Janouška', ' (132 m)']",['12 000 Kč za měsíc'],NaN,...,"['Albert Supermarket', ' (451 m)']","['Praha-Kyje', ' (1445 m)']",['Osobní'],NaN,NaN,NaN,NaN,NaN,"['ZUŠ Praha 9, Ratibořická', ' (234 m)']","['MŠ Sluníčko Praha 9 - Černý Most', ' (279 m)']"
1,0,"Bořivojova, Praha 3 - Žižkov Panorama",['Dnes'],NaN,"['Bankomat České spořitelny', ' (122 m)']",NaN,"[""'boolean-false'""]","['Tachovské náměstí', ' (443 m)']","['9 500 Kč za nemovitost, + provize RK']",NaN,...,"['Shalamar Foods', ' (165 m)']","['Praha hlavní nádraží', ' (984 m)']",['Osobní'],NaN,NaN,NaN,NaN,NaN,"['Gymnázium Karla Sladkovského Praha 3', ' (56...","['ZŠ a MŠ Jaroslava Seiferta Praha 3', ' (243 ..."
2,0,"Tupolevova, Praha 9 - Letňany Panorama",['Dnes'],NaN,"['Bankomat České spořitelny', ' (236 m)']",NaN,"[""'boolean-true'""]","['Šumperská', ' (177 m)']",['13 900 Kč za měsíc'],NaN,...,"['Penny Market', ' (183 m)']","['Praha-Čakovice', ' (1695 m)']",['Osobní'],NaN,NaN,"[""'boolean-true'""]",NaN,NaN,"['Základní škola Fryčovická', ' (160 m)']","['MŠ Havířovská', ' (255 m)']"
3,0,"Podbělohorská, Praha 5 - Smíchov Panorama",['Dnes'],NaN,"['Bankomat Komerční banky', ' (381 m)']",NaN,"[""'boolean-false'""]","['Pod Lipkami', ' (30 m)']","['15 000 Kč za nemovitost, + provize RK']",NaN,...,"['Smíšené zboží Plzeňská\t187', ' (404 m)']","['Praha-Cibulka', ' (1229 m)']",['Osobní'],['Dálkový vodovod'],NaN,NaN,NaN,NaN,"['ZUŠ Na Popelce', ' (224 m)']","['TROIS PETITES POMMES, s.r.o.', ' (343 m)']"


In [3]:
data = data.drop(["Unnamed: 0"], axis= 1)  #this drops the unnamed column as it has no value
data.info() #inspecting all the variables, their type and number of observations

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7878 entries, 0 to 7877
Data columns (total 79 columns):
Adresa                                     7878 non-null object
Aktualizace:                               7878 non-null object
Balkón:                                    1960 non-null object
Bankomat:                                  7455 non-null object
Bazén:                                     21 non-null object
Bezbariérový:                              1744 non-null object
Bus MHD:                                   7756 non-null object
Celková cena:                              7646 non-null object
Cena:                                      47 non-null object
Cukrárna:                                  7455 non-null object
Datum nastěhování:                         1751 non-null object
Datum prohlídky do:                        5 non-null object
Datum prohlídky:                           46 non-null object
Datum zahájení prodeje:                    8 non-null object
Divadlo: 

In [4]:
#We rename columns so that we can better work with them 
#Specifically, adding underscored characters in multi-word names, deleting colons, 
#rewriting some long names in shorter form (among those that are very likely to be used in the analysis)

data.columns = ['Adresa', 'Aktualizace', 'Balkón', 'Bankomat', 'Bazén',
       'Bezbariérový', 'Bus_MHD', 'Price_total', 'Price', 'Cukrárna',
       'Datum_nastěhování', 'Datum_prohlídky_do', 'Datum_prohlídky',
       'Datum_zahájení_prodeje', 'Divadlo', 'Doprava', 'Elektřina',
       'Energy_demands', 'Garáž', 'Hospoda', 'Hřiště',
       'ID_zakázky:', 'ID', 'Kino', 'Komunikace', 'Kulturní_památka',
       'Lodžie', 'Lékař', 'Lékárna', 'Metro', 'Náklady_na_bydlení',
       'Obchod', 'Odpad', 'Parkování', 'Plocha_podlahová',
       'Plocha_pozemku', 'Plocha_zahrady', 'Plocha_zastavěná', 'Plyn',
       'Podlaží', 'Poloha domu', 'Poznámka_k_ceně', 'Počet bytů', 'Pošta',
       'Provize', 'Energy_demands_II', 'Převod_do_OV',
       'Přírodní zajímavost', 'Půdní_vestavba', 'Původní_cena',
       'Restaurace', 'Rok_kolaudace', 'Rok_rekonstrukce', 'Sklep',
       'Sportoviště', 'Stav_objektu', 'Stav', 'Stavba', 'Telekomunikace',
       'Terasa', 'Title', 'Topení', 'Tram', 'Typ_bytu', 'Typ_domu',
       'Energy_demands_III', 'Umístění_objektu',
       'Area', 'Veterinář', 'Večerka', 'Vlak', 'Vlastnictví',
       'Voda', 'Vybavení', 'Výtah', 'Výška_stropu', 'Zlevněno', 'Škola',
       'Školka']

In total, we have 7878 observations and 79 columns. Neverheless, as we can see above, many of the columns contain only a relatively limited information (in terms of number of their observations), specifically 49 variables have more than 10% of missing values. Hence, the strenght of their evidence is limited and we need to be aware of this limitation. Variables with extreme portion of missing values (> 90%) deserve special attention and we should rather not include them in the analysis at all because they don't tell us much.

In [5]:
n_observations = 7878
empty = data.isnull().sum()
empty[empty > n_observations*0.9].sort_values(ascending = False).count()
#number of variables that miss more than 90% of observations

25

In [6]:
empty[empty > n_observations*0.9].sort_values(ascending = False) #list of those variables

Typ_domu                  7877
Plocha_pozemku            7877
Poloha domu               7877
Datum_prohlídky_do        7873
Datum_zahájení_prodeje    7870
Počet bytů                7870
Bazén                     7857
Převod_do_OV              7854
Půdní_vestavba            7838
Datum_prohlídky           7832
Price                     7831
Stav                      7818
Plocha_zahrady            7808
Výška_stropu              7801
Energy_demands_II         7719
Provize                   7705
Původní_cena              7693
Zlevněno                  7693
Plocha_zastavěná          7663
Rok_kolaudace             7643
Energy_demands_III        7579
Typ_bytu                  7547
Rok_rekonstrukce          7319
Lodžie                    7309
Náklady_na_bydlení        7224
dtype: int64

The most suitable variables are these 31 that miss less than 90% of observations. Regarding the aim of our project, it is pleasant that variable 'Metro' can be found here. In our analysis, we will consider 6 more variables, i.e. all variables that contain more than 66% of observations.

In [7]:
empty[empty < n_observations*0.1].sort_values(ascending = False)

Kulturní_památka    757
Kino                463
Divadlo             459
Obchod              456
Restaurace          423
Bankomat            423
Cukrárna            423
Hospoda             423
Lékař               423
Lékárna             423
Škola               423
Pošta               423
Školka              423
Veterinář           423
Večerka             423
Metro               254
Price_total         232
Tram                205
Vlak                128
Bus_MHD             122
Sportoviště         117
Hřiště              117
Vlastnictví           1
Podlaží               0
Area                  0
Title                 0
Stavba                0
Stav_objektu          0
Aktualizace           0
Adresa                0
dtype: int64

In [8]:
empty[np.all([(empty > n_observations*0.1),(empty < n_observations*0.33)], axis = 0)].sort_values(ascending = False)

Plocha_podlahová    2575
Výtah               2249
Poznámka_k_ceně     2114
Vybavení            1901
Energy_demands      1896
ID_zakázky:          953
dtype: int64

## II. Dataset adjustments
At this point, two things need to be done. Adjusting the dataset only for variables that have some evidence power, and adjusting the column 'Price' as its string values are currently very messy which makes it impossible to work with this crucial variable in our analysis.

### a) Keeping worthy variables only
As discussed above, we consider only variables with more than 66% of observations. Let's adjust the dataset accordingly...

In [9]:
variable = empty[empty < n_observations*0.33]
variable.index

Index(['Adresa', 'Aktualizace', 'Bankomat', 'Bus_MHD', 'Price_total',
       'Cukrárna', 'Divadlo', 'Energy_demands', 'Hospoda', 'Hřiště',
       'ID_zakázky:', 'Kino', 'Kulturní_památka', 'Lékař', 'Lékárna', 'Metro',
       'Obchod', 'Plocha_podlahová', 'Podlaží', 'Poznámka_k_ceně', 'Pošta',
       'Restaurace', 'Sportoviště', 'Stav_objektu', 'Stavba', 'Title', 'Tram',
       'Area', 'Veterinář', 'Večerka', 'Vlak', 'Vlastnictví', 'Vybavení',
       'Výtah', 'Škola', 'Školka'],
      dtype='object')

In [10]:
data = data[variable.index]

In [11]:
data[variable.index].head()

,Adresa,Aktualizace,Bankomat,Bus_MHD,Price_total,Cukrárna,Divadlo,Energy_demands,Hospoda,Hřiště,...,Tram,Area,Veterinář,Večerka,Vlak,Vlastnictví,Vybavení,Výtah,Škola,Školka
0,"Dobrovolného, Praha 9 - Černý Most Panorama",['Dnes'],"['Bankomat České spořitelny', ' (542 m)']","['Generála Janouška', ' (132 m)']",['12 000 Kč za měsíc'],"['Shisha Room Dvin', ' (1144 m)']","['Přírodní divadlo Dády Stoklasy', ' (3152 m)']",['Třída G - Mimořádně nehospodárná'],"['Šenk Na Rajské', ' (476 m)']","['Dětské hřiště Rajský vrch', ' (162 m)']",...,"['Lehovec', ' (1136 m)']",['26m2'],"['MetropoleVet Praha s.r.o.', ' (623 m)']","['Albert Supermarket', ' (451 m)']","['Praha-Kyje', ' (1445 m)']",['Osobní'],NaN,NaN,"['ZUŠ Praha 9, Ratibořická', ' (234 m)']","['MŠ Sluníčko Praha 9 - Černý Most', ' (279 m)']"
1,"Bořivojova, Praha 3 - Žižkov Panorama",['Dnes'],"['Bankomat České spořitelny', ' (122 m)']","['Tachovské náměstí', ' (443 m)']","['9 500 Kč za nemovitost, + provize RK']","['Klub JINÝ KAFE', ' (283 m)']","['Žižkovské divadlo Járy Cimrmana', ' (280 m)']",['Třída G - Mimořádně nehospodárná'],"['Hospůdka U Habásků', ' (209 m)']","['Dětské hřiště Bořivojova', ' (294 m)']",...,"['Lipanská', ' (186 m)']",['23m2'],"['MVDr. Jan Dubský', ' (425 m)']","['Shalamar Foods', ' (165 m)']","['Praha hlavní nádraží', ' (984 m)']",['Osobní'],NaN,NaN,"['Gymnázium Karla Sladkovského Praha 3', ' (56...","['ZŠ a MŠ Jaroslava Seiferta Praha 3', ' (243 ..."
2,"Tupolevova, Praha 9 - Letňany Panorama",['Dnes'],"['Bankomat České spořitelny', ' (236 m)']","['Šumperská', ' (177 m)']",['13 900 Kč za měsíc'],"['Café Infinity', ' (204 m)']","['Divadlo Pohádka', ' (1057 m)']",NaN,"['Pivnice na Královce', ' (1824 m)']","['Venkovní posilovna u obchodního centra', ' (...",...,"['Sídliště Ďáblice', ' (1906 m)']",['52m2'],"['Veterinární ordinace Letňany', ' (302 m)']","['Penny Market', ' (183 m)']","['Praha-Čakovice', ' (1695 m)']",['Osobní'],NaN,"[""'boolean-true'""]","['Základní škola Fryčovická', ' (160 m)']","['MŠ Havířovská', ' (255 m)']"
3,"Podbělohorská, Praha 5 - Smíchov Panorama",['Dnes'],"['Bankomat Komerční banky', ' (381 m)']","['Pod Lipkami', ' (30 m)']","['15 000 Kč za nemovitost, + provize RK']","['Cukrárna Pod Marjánkou', ' (1333 m)']","['JUDr. Jiří Šilhán', ' (279 m)']",['Třída B - Velmi úsporná'],"['Restaurace Na zámyšli', ' (423 m)']","['Dětské hřiště Na Vršku', ' (116 m)']",...,"['Kavalírka', ' (370 m)']",['58m2'],"['Veterinární klinika - MVDr. Pavel Santar, MV...","['Smíšené zboží Plzeňská\t187', ' (404 m)']","['Praha-Cibulka', ' (1229 m)']",['Osobní'],NaN,NaN,"['ZUŠ Na Popelce', ' (224 m)']","['TROIS PETITES POMMES, s.r.o.', ' (343 m)']"
4,"Mezi domy, Praha 4 - Písnice Panorama",['Dnes'],"['Bankomat Komerční banky', ' (780 m)']","['U Libušské sokolovny', ' (275 m)']",['19 500 Kč za měsíc'],"['Cukrářství Viktoria, s.r.o.', ' (900 m)']","['Tradiční loutkové divadlo Zvoneček', ' (3145...",['Třída D - Méně úsporná č. 78/2013 Sb. podle ...,"['Hospůdka U Báti', ' (509 m)']","['Dětské hřiště Pyramida Na Okruhu', ' (78 m)']",...,"['Levského', ' (1815 m)']",['93m2'],"['Vetnemo, s.r.o.', ' (634 m)']","['VIVACOMEX, spol. s r.o.', ' (450 m)']","['Praha-Krč', ' (3635 m)']",['Osobní'],NaN,"[""'boolean-true'""]",['Střední odborné učiliště potravinářské Praha...,"['Mateřská škola Mezi Domy', ' (105 m)']"


In [12]:
data[variable.index].head().to_csv('dataview.csv', sep = ';')

# Data cleaning and processing

## Cleaning the square brackets in the whole dataset

In [13]:
data = data.applymap(lambda x: x.strip("['']") if type(x) == str else x) #deletes all the square brackets if type string

### b) Dealing with messy prices (alternative 1 - Sila advice)

Before we can do the analysis, we need to deal with the complicated form of stated prices and obtain purely the monthly rent in CZK, without other specification (in other words, we need to extract integer out of the string values).

In order to be able to suitably merge the resulting list of prices to the original dataset, it is neccessary to get rid of rows with empty prices values beforehand. Furthermore, during the analysis, we explored two error values in the dataset (tiny but with serious consequences): observation data.Price_total[984] has value of "12 CZK" which is nonsense and observation data.Price_total[6727] has value "23 214 CZK (850 EUR)" which cases troubles in the analysis and we need to get rid of both of these errors beforehand.

In [14]:
def get_price(x):
    if ~(type(x) == float and np.isnan(x)): #this check if a valus is not an NA
        kc = x.split('Kč') #splits the string by Kc
        kc_nogap = kc[0].replace(' ','') #gets the string left of Kc and deletes spaces
        kc_int = int(kc_nogap) #converts to int
        return kc_int 
    else:
        return x #if NA stay NA

In [15]:
data['Cena'] = data['Price_total'].apply(get_price) #apply the above defined function to column Price_total

In [16]:
data['Cena']

0       12000.0
1        9500.0
2       13900.0
3       15000.0
4       19500.0
         ...   
7873    35000.0
7874    33000.0
7875    55000.0
7876    75000.0
7877    35000.0
Name: Cena, Length: 7878, dtype: float64

### b) Dealing with messy prices (alternative 3 - split method)

In [17]:
prices_strings = [str(x) for x in data.Price_total]

In [18]:
prices_alt = []

for x in prices_strings:
    prices_alt.append(x.split("Kč")[0].replace(" ","").replace("'","").replace("[",""))

print(prices_alt)

['12000', '9500', '13900', '15000', '19500', '17000', '12500', '8500', '13000', '34900', '13000', '15200', '29000', '13490', '14500', '14500', '19500', '13000', '18000', '17000', '17500', '15500', '7500', '12500', '27000', '20000', '15000', 'nan', 'nan', '15500', '20000', '14800', '110000', '39000', '19000', '14700', '18000', '16000', '18900', '11990', '25000', '13500', '14500', '21000', '17000', '35000', '30000', '60497', '43000', '35000', '99500', '49500', '49000', '33000', '24000', '13000', '12000', '15288', '9500', '10600', '14500', '9000', '18800', '11500', '15000', '9950', '16000', '14450', '8450', '8000', '12000', '16500', '14500', 'nan', '12250', '13900', '17500', 'nan', '9450', '9500', '12000', '9400', '9000', '11500', '25000', '14000', '15500', '15000', '14800', '12200', '19000', '14000', '7500', '14500', '14000', '16000', '25500', '13500', '15000', '13250', '14000', '14000', '12500', '15500', '21000', '17500', '22000', '17000', '13500', '20000', '10500', '14000', '13000', '8

In [19]:
prices_alt_float = [float(i) for i in prices_alt]
len(prices_alt_float)

7878

In [20]:
prices_alt_float[3993]

126992.0

Number of NaNs:

In [21]:
YZX = np.array(prices_alt_float)
NaNs_alt = np.argwhere(np.isnan(YZX))
len(NaNs_alt)

232

### Separate distance and name  in variable containing 'list'

In [22]:
import re
def extractor(inp): 
    ''' This function deletes all non numeric chachters from a string
    '''
    try: 
        result = int(re.sub("[^0-9]", "",inp))
    except:
        result = None
    return result

In [23]:
def brush(x, posi):
    ''' 
    This function solves one problem with a structure of our data, we stored it as a list but pandas converted it to string.
    We therefore grab the values by "," in a list structure [" "," "]. We know our lists have two values at most
    '''
    if ~(type(x) == float and np.isnan(x)):
        try:
            first = x.strip('[]') # this line is probably redundand since we cleaned the dataset above 
            return first.split("', '")[posi] 
        except IndexError: #in column Restaurant some values have a different structure
            first = x.strip('[]') # this line is probably redundand since we cleaned the dataset above 
            return first.split(",")[posi] 
    else:
        return None  #once NA always NA

In [24]:
def clean(var_name): #from our intitial list we create two new columns 
    data[f'{var_name}_name'] = data[var_name].apply(brush, args = (0,))
    data[f'{var_name}_dist'] = data[var_name].apply(brush, args = (1,))
    data[f'{var_name}_dist'] = data[f'{var_name}_dist'].apply(extractor) #we keep only digits converted to int in the _dist column
    

In [25]:
var_to_clean = ['Bankomat','Bus_MHD', 'Cukrárna','Divadlo','Hospoda','Hřiště','Kino',
 'Kulturní_památka', 'Lékař', 'Lékárna', 'Metro','Obchod','Pošta','Restaurace', 'Sportoviště',
'Tram', 'Veterinář', 'Večerka', 'Vlak','Škola', 'Školka'] # list of column names to clean

In [26]:
for var in var_to_clean:
    clean(var)

### converting True & False to 1 & 0

In [27]:
def Booli(x): # converts special true and false strings to  1 & 0
    if ~(type(x) == float and np.isnan(x)):
        if x == '"\'boolean-false\'"':
            return 0
        elif x == '"\'boolean-true\'"':
            return 1
        else:
            return x
    else:
        return x

In [28]:
def convert_01(var_name): #rewrites values by a Booli function
    data[var_name] = data[var_name].apply(Booli)  

In [29]:
var_conv = ['Výtah','Vybavení']

In [30]:
for var in var_conv:
    convert_01(var)

### Analyzing certain vars

In [31]:
def group_exm(var_name): #this function returns values and they frequency of occurence
    return data.groupby(var_name)[var_name].count()  

In [32]:
group_exm('Aktualizace')

Aktualizace
01.02.2020       1
01.03.2020       1
01.04.2020      71
02.01.2020       2
02.02.2020       5
              ... 
30.03.2020      46
30.11.2019       1
31.03.2020     104
Dnes          1271
Včera          489
Name: Aktualizace, Length: 127, dtype: int64

In [33]:
group_exm('Stav_objektu')

Stav_objektu
Dobrý                 630
K demolici              1
Novostavba           1267
Po rekonstrukci      1679
Projekt                 2
Před rekonstrukcí      11
Ve výstavbě             1
Velmi dobrý          4287
Name: Stav_objektu, dtype: int64

In [34]:
group_exm('Stavba')

Stavba
Cihlová      6365
Dřevěná         6
Kamenná        10
Montovaná       8
Panelová      635
Skeletová     243
Smíšená       611
Name: Stavba, dtype: int64

In [35]:
group_exm('Vlastnictví')

Vlastnictví
Družstevní        208
Osobní           7633
Státní/obecní      36
Name: Vlastnictví, dtype: int64

In [36]:
group_exm('Vybavení')

Vybavení
0            987
1           3845
Částečně    1145
Name: Vybavení, dtype: int64

In [37]:
group_exm('Energy_demands') #this could yet tell us something

Energy_demands
Třída A - Mimořádně úsporná                                          42
Třída A - Mimořádně úsporná č. 148/2007 Sb. podle vyhlášky            1
Třída A - Mimořádně úsporná č. 78/2013 Sb. podle vyhlášky            11
Třída B - Velmi úsporná                                             298
Třída B - Velmi úsporná č. 148/2007 Sb. podle vyhlášky               34
Třída B - Velmi úsporná č. 78/2013 Sb. podle vyhlášky               284
Třída C - Úsporná                                                   477
Třída C - Úsporná č. 148/2007 Sb. podle vyhlášky                     49
Třída C - Úsporná č. 78/2013 Sb. podle vyhlášky                     341
Třída D - Méně úsporná                                              187
Třída D - Méně úsporná č. 148/2007 Sb. podle vyhlášky                21
Třída D - Méně úsporná č. 78/2013 Sb. podle vyhlášky                187
Třída E - Nehospodárná                                               51
Třída E - Nehospodárná č. 148/2007 Sb. podle vyhl

### Transformation of energy demands

In [38]:
def eng_class(x):
    
    if x in ['Třída A - Mimořádně úsporná','Třída A - Mimořádně úsporná č. 148/2007 Sb. podle vyhlášky','Třída A - Mimořádně úsporná č. 78/2013 Sb. podle vyhlášky']:
        clas = 'A'
    elif x in ['Třída B - Velmi úsporná', 'Třída B - Velmi úsporná č. 148/2007 Sb. podle vyhlášky', 'Třída B - Velmi úsporná č. 78/2013 Sb. podle vyhlášky']:
        clas = 'B'
    elif x in ['Třída C - Úsporná', 'Třída C - Úsporná č. 148/2007 Sb. podle vyhlášky', 'Třída C - Úsporná č. 78/2013 Sb. podle vyhlášky']:
        clas = 'C'
    elif x in ['Třída D - Méně úsporná', 'Třída D - Méně úsporná č. 148/2007 Sb. podle vyhlášky', 'Třída D - Méně úsporná č. 78/2013 Sb. podle vyhlášky']:
        clas = 'D'
    elif x in ['Třída E - Nehospodárná','Třída E - Nehospodárná č. 148/2007 Sb. podle vyhlášky', 'Třída E - Nehospodárná č. 78/2013 Sb. podle vyhlášky']:
        clas = 'E'
    elif x in ['Třída F - Velmi nehospodárná','Třída F - Velmi nehospodárná č. 78/2013 Sb. podle vyhlášky']:
        clas = 'F'
    elif x in ['Třída G - Mimořádně nehospodárná','Třída G - Mimořádně nehospodárná č. 148/2007 Sb. podle vyhlášky', 'Třída G - Mimořádně nehospodárná č. 78/2013 Sb. podle vyhlášky']:
        clas = 'G'
    else:
        clas = x
    
    return clas

In [39]:
data['Energy_class'] = data['Energy_demands'].apply(eng_class)

In [40]:
group_exm('Energy_class')

Energy_class
A      54
B     616
C     867
D     395
E     120
F      47
G    3883
Name: Energy_class, dtype: int64

### Transformation of Plocha_podlahova and Area

In [41]:
ar_var = ['Area', 'Plocha_podlahová']

In [42]:
def ext_m2(inp): #converting area based variable to int
    if ~(type(inp) == float and np.isnan(inp)):
        return int(inp.split('m2')[0])
    else:
        return inp

In [43]:
for var in ar_var:
    data[var] = data[var].apply(ext_m2)

In [44]:
sum(data['Plocha_podlahová'] == data['Area']) #those variables are the same in 50% of the cases

4593

In [45]:
#sloupce ID_zakáyzky, Aktualize asi nemaji velkou informacni hodnotu pro nase ucely. 

### Extracting value from a Title

In [46]:
def get_rooms(x): #This function extract the number of rooms(type) from the title
    words = x.split(' ')
    for word in words:
        if '+' in word:
            return(word)
        elif 'atypické' == word:
            return('atyp')
        elif 'pokoje' == word:
            return(1)
        elif 'více' == word:
            return('6+')

In [47]:
data['Rooms'] = data['Title'].apply(get_rooms)

In [48]:
group_exm('Rooms') #tohle rika, ze Prazaci maji vetsinou oddeleny zachod a koupelnu

Rooms
1         83
1+1      395
1+kk    1476
2+1      701
2+kk    2477
3+1      649
3+kk    1271
4+1      222
4+kk     394
5+1       65
5+kk      74
6+        37
atyp      33
Name: Rooms, dtype: int64

### Patro

In [53]:
data['Podlaží_č'] = data['Podlaží'].apply(lambda x: x[0]) #This extract the floor(always first) from Podlazi

## Number of NaNs:

In [55]:
XYZ = np.array(data.Cena)
NaNs = np.argwhere(np.isnan(XYZ))
len(NaNs)

232

### c) Dealing with messy adresses

In this case, regex method extracting numerical characters (=districts of Prague) from observations appears to be the best way. Using the split fce doesn't seem to work well in this case.

In [56]:
#my first attempts for the split method. does not work that nice, but i keep it here for case...
#adress_strings = [str(i) for i in data.Adresa]

In [57]:
#district = []

#for y in adress_strings:
#    district.append(y.split("-")[0][-8:].replace(" ","").replace("[",""))

#print(district)

In [58]:
district = data['Adresa'].apply(extractor)
print(district)

0        9.0
1        3.0
2        9.0
3        5.0
4        4.0
        ... 
7873     NaN
7874    10.0
7875    10.0
7876     7.0
7877     7.0
Name: Adresa, Length: 7878, dtype: float64


In [59]:
data['district'] = district

In [60]:
data['district'].sort_values(ascending = False)[1:31] 
#we can see that these values apparently doesn't give as what we want. 
#it is because of occurence of "28. října" in couple of adresses of Prague 1 and "28. pluku" in Prague 10,
#"5. máje" in Prague 5, "5.května" in Prague 4, "obětí 6.května" in Prague 4 and "8.listopadu" in Prague 6.
#For these adresses we need to get rid of the first 1 or 2 characters to obtain the districts' numbers.
#Furthermore also 'Praha 10 - Praha 15 Panorama' [index 5017] and 'Praha 4 - Praha 11 Panorama' [index 2750]
#bring some complications. For these two, we need to get rid of last 2 characters.

1098    2810.0
2489    2810.0
6459    2810.0
83      2810.0
1436    2810.0
2165    2810.0
1693    2810.0
3550    2810.0
6768    2810.0
2819    2810.0
2488    2810.0
5017    1015.0
2750     411.0
4575     281.0
7230     281.0
7783     281.0
4698     281.0
7023     281.0
6192     281.0
168       86.0
74        64.0
7607      55.0
6107      55.0
651       55.0
6216      54.0
236       54.0
3223      54.0
134       54.0
1800      54.0
3956      14.0
Name: district, dtype: float64

In [61]:
data['Adresa'][168]

'8. listopadu, Praha 6 - Břevnov'

In [62]:
fal_indx = data['district'].sort_values(ascending = False)[1:30].index

In [68]:
dist_name = ('Panorama', 'Praha1','Praha2','Praha3','Praha4','Praha5','Praha6','Praha7','Praha8','Praha9','Praha10','Praha11','Praha12','Praha13','Praha14','Praha15','Praha16','Praha17','Praha18','Praha19','Praha20','Praha21','Praha22')

In [69]:
a = [district.count(x) for x in dist_name]
print(a)

KeyError: 'Requested level (Praha1) does not match index name (None)'

In [138]:
55+1300+997+731+718+1131+491+313+524+488

6748

### Let us reconsider which var are valuable

In [44]:
pd.set_option('display.max_columns', None)
data.head(4)

,Adresa,Aktualizace,Bankomat,Bus_MHD,Price_total,Cukrárna,Divadlo,Energy_demands,Hospoda,Hřiště,ID_zakázky:,Kino,Kulturní_památka,Lékař,Lékárna,Metro,Obchod,Plocha_podlahová,Podlaží,Poznámka_k_ceně,Pošta,Restaurace,Sportoviště,Stav_objektu,Stavba,Title,Tram,Area,Veterinář,Večerka,Vlak,Vlastnictví,Vybavení,Výtah,Škola,Školka,Cena,Bankomat_name,Bankomat_dist,Bus_MHD_name,Bus_MHD_dist,Cukrárna_name,Cukrárna_dist,Divadlo_name,Divadlo_dist,Hospoda_name,Hospoda_dist,Hřiště_name,Hřiště_dist,Kino_name,Kino_dist,Kulturní_památka_name,Kulturní_památka_dist,Lékař_name,Lékař_dist,Lékárna_name,Lékárna_dist,Metro_name,Metro_dist,Obchod_name,Obchod_dist,Pošta_name,Pošta_dist,Restaurace_name,Restaurace_dist,Sportoviště_name,Sportoviště_dist,Tram_name,Tram_dist,Veterinář_name,Veterinář_dist,Večerka_name,Večerka_dist,Vlak_name,Vlak_dist,Škola_name,Škola_dist,Školka_name,Školka_dist,Energy_class,Rooms,Podlaží_č
0,"Dobrovolného, Praha 9 - Černý Most Panorama",Dnes,"Bankomat České spořitelny', ' (542 m)","Generála Janouška', ' (132 m)",12 000 Kč za měsíc,"Shisha Room Dvin', ' (1144 m)","Přírodní divadlo Dády Stoklasy', ' (3152 m)",Třída G - Mimořádně nehospodárná,"Šenk Na Rajské', ' (476 m)","Dětské hřiště Rajský vrch', ' (162 m)",N05462,"CineStar Černý Most', ' (1815 m)","Hajnova vila', ' (3984 m)","MUDr. Zdeněk Valentík', ' (363 m)","Lékárna U Rajské zahrady', ' (500 m)","Rajská zahrada', ' (525 m)","Kaufland', ' (1486 m)",26.0,1. podlaží z celkem 5,"+ 2.500,-Kč poplatky, elektřina, provize RK","Pošta Praha 98 - Česká pošta, s.p.', ' (442 m)","Jiří Stopfer', ' (443 m)","Veřejné hřiště na míčové hry Pospíchalova', ' ...",Novostavba,Skeletová,Pronájem bytu 1+kk 26 m²,"Lehovec', ' (1136 m)",26,"MetropoleVet Praha s.r.o.', ' (623 m)","Albert Supermarket', ' (451 m)","Praha-Kyje', ' (1445 m)",Osobní,NaN,NaN,"ZUŠ Praha 9, Ratibořická', ' (234 m)","MŠ Sluníčko Praha 9 - Černý Most', ' (279 m)",12000.0,Bankomat České spořitelny,542.0,Generála Janouška,132.0,Shisha Room Dvin,1144.0,Přírodní divadlo Dády Stoklasy,3152.0,Šenk Na Rajské,476.0,Dětské hřiště Rajský vrch,162.0,CineStar Černý Most,1815.0,Hajnova vila,3984.0,MUDr. Zdeněk Valentík,363.0,Lékárna U Rajské zahrady,500.0,Rajská zahrada,525.0,Kaufland,1486.0,"Pošta Praha 98 - Česká pošta, s.p.",442.0,Jiří Stopfer,443.0,Veřejné hřiště na míčové hry Pospíchalova,295.0,Lehovec,1136.0,MetropoleVet Praha s.r.o.,623.0,Albert Supermarket,451.0,Praha-Kyje,1445.0,"ZUŠ Praha 9, Ratibořická",234.0,MŠ Sluníčko Praha 9 - Černý Most,279.0,G,1+kk,1
1,"Bořivojova, Praha 3 - Žižkov Panorama",Dnes,"Bankomat České spořitelny', ' (122 m)","Tachovské náměstí', ' (443 m)","9 500 Kč za nemovitost, + provize RK","Klub JINÝ KAFE', ' (283 m)","Žižkovské divadlo Járy Cimrmana', ' (280 m)",Třída G - Mimořádně nehospodárná,"Hospůdka U Habásků', ' (209 m)","Dětské hřiště Bořivojova', ' (294 m)",57/4562,"Filmový klub VŠE', ' (691 m)","Dům U černé Matky Boží', ' (1846 m)","MUDr. Milan Hudi', ' (502 m)","Lékárna U Matky Boží', ' (147 m)","Jiřího z Poděbrad', ' (652 m)","PAVILON, a.s.', ' (941 m)",23.0,5. podlaží z celkem 4,"plus poplatky, 500","Pošta Praha 35 - Česká pošta, s.p.', ' (176 m)","Restaurace Lavička', ' (119 m)","Sportovní centrum Olšanka', ' (469 m)",Dobrý,Cihlová,Pronájem bytu 1+kk 23 m²,"Lipanská', ' (186 m)",23,"MVDr. Jan Dubský', ' (425 m)","Shalamar Foods', ' (165 m)","Praha hlavní nádraží', ' (984 m)",Osobní,NaN,NaN,"Gymnázium Karla Sladkovského Praha 3', ' (56 m)","ZŠ a MŠ Jaroslava Seiferta Praha 3', ' (243 m)",9500.0,Bankomat České spořitelny,122.0,Tachovské náměstí,443.0,Klub JINÝ KAFE,283.0,Žižkovské divadlo Járy Cimrmana,280.0,Hospůdka U Habásků,209.0,Dětské hřiště Bořivojova,294.0,Filmový klub VŠE,691.0,Dům U černé Matky Boží,1846.0,MUDr. Milan Hudi,502.0,Lékárna U Matky Boží,147.0,Jiřího z Poděbrad,652.0,"PAVILON, a.s.",941.0,"Pošta Praha 35 - Česká pošta, s.p.",176.0,Restaurace Lavička,119.0,Sportovní centrum Olšanka,469.0,Lipanská,186.0,MVDr. Jan Dubský,425.0,Shalamar Foods,165.0,Praha hlavní nádraží,984.

In [45]:
worth = ['Adresa', 'Aktualizace', 'Price_total','ID_zakázky:', 'Plocha_podlahová', 'Podlaží', 'Podlaží_č', 
'Poznámka_k_ceně', 'Stav_objektu', 'Stavba', 'Title','Rooms', 
'Area', 'Vlastnictví', 'Vybavení', 'Výtah','Bankomat_name', 'Bankomat_dist',
'Bus_MHD_name', 'Bus_MHD_dist', 'Cukrárna_name', 'Cukrárna_dist',
'Divadlo_name', 'Divadlo_dist', 'Hospoda_name', 'Hospoda_dist',
'Hřiště_name', 'Hřiště_dist', 'Kino_name', 'Kino_dist',
'Kulturní_památka_name', 'Kulturní_památka_dist', 'Lékař_name',
'Lékař_dist', 'Lékárna_name', 'Lékárna_dist', 'Metro_name',
'Metro_dist', 'Obchod_name', 'Obchod_dist', 'Pošta_name', 'Pošta_dist',
'Restaurace_name', 'Restaurace_dist', 'Sportoviště_name',
'Sportoviště_dist', 'Tram_name', 'Tram_dist', 'Veterinář_name',
'Veterinář_dist', 'Večerka_name', 'Večerka_dist', 'Vlak_name',
'Vlak_dist', 'Škola_name', 'Škola_dist', 'Školka_name', 'Školka_dist',
'Energy_class']

In [46]:
data1 = data[worth].copy()

In [47]:
pd.set_option('display.max_columns', None)
data1.head(4)

,Adresa,Aktualizace,Price_total,ID_zakázky:,Plocha_podlahová,Podlaží,Podlaží_č,Poznámka_k_ceně,Stav_objektu,Stavba,Title,Rooms,Area,Vlastnictví,Vybavení,Výtah,Bankomat_name,Bankomat_dist,Bus_MHD_name,Bus_MHD_dist,Cukrárna_name,Cukrárna_dist,Divadlo_name,Divadlo_dist,Hospoda_name,Hospoda_dist,Hřiště_name,Hřiště_dist,Kino_name,Kino_dist,Kulturní_památka_name,Kulturní_památka_dist,Lékař_name,Lékař_dist,Lékárna_name,Lékárna_dist,Metro_name,Metro_dist,Obchod_name,Obchod_dist,Pošta_name,Pošta_dist,Restaurace_name,Restaurace_dist,Sportoviště_name,Sportoviště_dist,Tram_name,Tram_dist,Veterinář_name,Veterinář_dist,Večerka_name,Večerka_dist,Vlak_name,Vlak_dist,Škola_name,Škola_dist,Školka_name,Školka_dist,Energy_class
0,"Dobrovolného, Praha 9 - Černý Most Panorama",Dnes,12 000 Kč za měsíc,N05462,26.0,1. podlaží z celkem 5,1,"+ 2.500,-Kč poplatky, elektřina, provize RK",Novostavba,Skeletová,Pronájem bytu 1+kk 26 m²,1+kk,26,Osobní,NaN,NaN,Bankomat České spořitelny,542.0,Generála Janouška,132.0,Shisha Room Dvin,1144.0,Přírodní divadlo Dády Stoklasy,3152.0,Šenk Na Rajské,476.0,Dětské hřiště Rajský vrch,162.0,CineStar Černý Most,1815.0,Hajnova vila,3984.0,MUDr. Zdeněk Valentík,363.0,Lékárna U Rajské zahrady,500.0,Rajská zahrada,525.0,Kaufland,1486.0,"Pošta Praha 98 - Česká pošta, s.p.",442.0,Jiří Stopfer,443.0,Veřejné hřiště na míčové hry Pospíchalova,295.0,Lehovec,1136.0,MetropoleVet Praha s.r.o.,623.0,Albert Supermarket,451.0,Praha-Kyje,1445.0,"ZUŠ Praha 9, Ratibořická",234.0,MŠ Sluníčko Praha 9 - Černý Most,279.0,G
1,"Bořivojova, Praha 3 - Žižkov Panorama",Dnes,"9 500 Kč za nemovitost, + provize RK",57/4562,23.0,5. podlaží z celkem 4,5,"plus poplatky, 500",Dobrý,Cihlová,Pronájem bytu 1+kk 23 m²,1+kk,23,Osobní,NaN,NaN,Bankomat České spořitelny,122.0,Tachovské náměstí,443.0,Klub JINÝ KAFE,283.0,Žižkovské divadlo Járy Cimrmana,280.0,Hospůdka U Habásků,209.0,Dětské hřiště Bořivojova,294.0,Filmový klub VŠE,691.0,Dům U černé Matky Boží,1846.0,MUDr. Milan Hudi,502.0,Lékárna U Matky Boží,147.0,Jiřího z Poděbrad,652.0,"PAVILON, a.s.",941.0,"Pošta Praha 35 - Česká pošta, s.p.",176.0,Restaurace Lavička,119.0,Sportovní centrum Olšanka,469.0,Lipanská,186.0,MVDr. Jan Dubský,425.0,Shalamar Foods,165.0,Praha hlavní nádraží,984.0,Gymnázium Karla Sladkovského Praha 3,56.0,ZŠ a MŠ Jaroslava Seiferta Praha 3,243.0,G
2,"Tupolevova, Praha 9 - Letňany Panorama",Dnes,13 900 Kč za měsíc,NaN,NaN,4. podlaží,4,NaN,Velmi dobrý,Panelová,Pronájem bytu 2+1 52 m²,2+1,52,Osobní,NaN,1.0,Bankomat České spořitelny,236.0,Šumperská,177.0,Café Infinity,204.0,Divadlo Pohádka,1057.0,Pivnice na Královce,1824.0,Venkovní posilovna u obchodního centra,194.0,Cinema City Letňany,223.0,Hajnova vila,2248.0,MUDr. Petr Zieg,317.0,"APOMED, s.r.o.",302.0,Letňany,1382.0,Tesco,366.0,"Pošta Praha 99 - Česká pošta, s.p.",680.0,Pub Infinity,207.0,Surf Arena,350.0,Sídliště Ďáblice,1906.0,Veterinární ordinace Letňany,302.0,Penny Market,183.0,Praha-Čakovice,1695.0,Základní škola Fryčovická,160.0,MŠ Havířovská,255.0,NaN
3,"Podbělohorská, Praha 5 - Smíchov Panorama",Dnes,"15 000 Kč za nemovitost, + provize RK",44/4562,58.0,2. podlaží z celkem 3,2,", 3500",Novostavba,Cihlová,Pronájem bytu 2+kk 58 m²,2+kk,58,Osobní,NaN,NaN,Bankomat Komerční banky,381.0,Pod Lipkami,30.0,Cukrárna Pod Marjánkou,1333.0,JUDr. Jiří Šilhán,279.0,Restaurace Na zámyšli,423.0,Dětské hřiště Na Vršku,116.0,CineStar Anděl,2038.0,Šaldova vila,757.0,MUDr. Drahomíra Miškovská,309.0,Lékárna Melissa,447.0,Radlická,2026.0,Obchodní centrum Nový Smíchov,2075.0,"Pošta Praha 56 - Česká pošta, s.p.",379.0,ŘÍZKÁRNA ROZVOZ JÍDEL,345.0,TJ Banka Praha z.s.,345.0,Kavalírka,370.0,"Veterinární klinika - MVDr. Pavel Santar, MVDr...",1141.0,Smíšené zboží Plzeňská\t187,404.0,Praha-Cibulka,1229.0,ZUŠ Na Popelce,224.0,"TROIS PETITES POMMES, s.r.o.",343.0,B


In [48]:
%store data1 #storing data1 to use in other jupyter notebooks

Stored 'data1' (DataFrame)
